![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Automated Curation Lab

**NOTE - this lab is still under construction and should not be used**

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information about data and technology (called metadata).

In the [Building a Data Catalog](building-a-data-catalog.ipynb) lab, Peter Profile and Erin Overivew
manually catalogued the weekly measurement files for the Drop Foot clinical trial.

In this hands-on lab you will get a chance to work with Egeria's governance servers to
automate this onboarding process.

## The scenario

[Coco Pharmaceuticals](https://opengovernance.odpi.org/coco-pharmaceuticals/)
is conducting a clinical trial with two hospitals: Oak Dene Hospital and Old Market Hospital.
Each week the two hospitals send Coco Pharmaceuticals a set of measurements from the patients
involved in the trial.  These measurements are located in a CSV file that the hospital sends through
secure file transfer to a folder in Coco Pharmaceutical's landing area.

These files need to be copied into the data lake and catalogued so that they are only visible to the
staff involved in the clinical trial.  It is also important that the lineage of these files is
maintained so the source of the data can be traced.  This process is shown in Figure 1.

![Scenario](../images/automated-curation-scenario.png)
> **Figure 1:** Clinical trial weekly measurements onboarding process

Peter Profile and Erin Overview are responsible for this onboarding process.
![Peter and Erin](../images/peter-and-erin.png)

They have defined a list of requirements for the process:

* Files must be in the landing area for a minimum amount of time.
* As a new file is received, it needs to be catalogued, including:
   * Description
   * Connection details to enable the data scientists to access the contents
   * Column details
   * Governance zones defining the files' visibility
   * Owner 
   * Origin
* A file is not accessible by any of the data lake users until the cataloguing process is complete.
* They must record lineage of each measurements file so they know which hospital it came from.

They have been [manually cataloguing the measurements files](building-a-data-catalog.ipynb) for
the first few weeks to prove the approach but now it is time to automate the process since:
* This clinical trial is planned to run for two years.
* There is a expected to be a ramp up of other clinical trials running simultaneously and the
  file onboarding workload would soon become overwhelming if they continued with the manual approach.

They plan to use an
[Integration Daemon](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/integration-daemon.html)
called **exchangeDL01** to capture the technical metadata of the files.
Then the 
[Engine Host](https://egeria.odpi.org/open-metadata-implementation/admin-services/docs/concepts/engine-host.html)
server called **governDL01** will manage the move of the file into the data lake,
the augmentation of the metadata properties of the files and the creation of the lineage.

This lab sets up the automated onbourding process using 6 phases as shown in Figure 2:

![Phases](../images/automated-curation-scenario-6.png)
> **Figure 2:** All six phases

1. Bring files in from "outside" using the move/copy file governance action service
2. Create a template used to catalog files in the landing areas
3. Trigger a governance action process when a file appears in a landing folder
4. Move the file into the data lake folder using a governance action process
5. Create a template used to catalog files in the data lake folder
6. Enrich the metadata using governance action services


## Setting up

Coco Pharmaceuticals make widespread use of Egeria for tracking and managing their data and related assets.
Figure 3 below shows their servers and the Open Metadata and Governance (OMAG) Server Platforms that are hosting them.

![Figure 3](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 3:** Coco Pharmaceuticals' OMAG Server Platforms

The code below checks that the platforms are running.  It checks that the servers are configured and whether they are running on the platform.  If a server is configured, but not running, it will start it.

Look for the second "Done." message that is displayed just after the governance servers have started.  It may take up to a minute to start up all of the servers.  If `cocoMDS2` seems to be very slow starting, check that Apache Kafka is running.  The metadata servers will wait for Kafka during start up if they are connected to a cohort.  When they are waiting in this manner, they periodically output a message on the console.

In [15]:
# Start up the metadata servers and the view server
%run ../common/environment-check.ipynb

print("Start up the Governance Servers")
activatePlatform(dataLakePlatformName, dataLakePlatformURL, [governDL01Name, exchangeDL01Name])

print("Done. ")


Checking OMAG Server Platform availability...
    Core Platform is active
        Checking OMAG Server cocoMDS2
           ... cocoMDS2 is configured
           ... cocoMDS2 is not known - needs to be started
                Starting server cocoMDS2 ...
        Checking OMAG Server cocoMDS3
           ... cocoMDS3 is configured
           ... cocoMDS3 is not known - needs to be started
                Starting server cocoMDS3 ...
        Checking OMAG Server cocoMDS5
           ... cocoMDS5 is configured
           ... cocoMDS5 is not known - needs to be started
                Starting server cocoMDS5 ...
        Checking OMAG Server cocoMDS6
           ... cocoMDS6 is configured
           ... cocoMDS6 is not known - needs to be started
                Starting server cocoMDS6 ...
    Data Lake Platform is active
        Checking OMAG Server cocoMDS1
           ... cocoMDS1 is configured
           ... cocoMDS1 is not known - needs to be started
                Starting server cocoM

----
You should see that both the metadata servers `cocoMDS1` and `cocoMDS2` along with the integration daemon `exchangeDL01` and the engine host server `governDL01` have started.  These are the servers that will be used in this lab and they are connected together
as shown in figure 4.  The `cocoMDS1` metadata server is where the new files are catalogued, whereas the definitions used
to drive the governance processes come from `cocoMDS2`.  

![Figure 4](../images/governance-servers-for-automated-onboarding.png)
> **Figure 4:** Governance servers for automated curation

If any of the platforms are not running, follow [this link to set up and run the platform](https://egeria.odpi.org/open-metadata-resources/open-metadata-labs/).  If any server is reporting that it is not configured then
run the steps in the [Server Configuration](../egeria-server-config.ipynb) lab to configure
the servers.  Then re-run the previous step to ensure all of the servers are started.

## Review the status of the integration daemon

At this point, even though both `exchangeDL01` and `governDL01` are running, there is still work to set up the full data onboarding pipeline.  Lets start with the integration daemon that catalogs the files as they appear in a folder.

The command below queries the status of the integration daemon called `exchangeDL01`.

In [ ]:

getIntegrationDaemonStatus(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId)


----
Notice that the Files Integrator Open Metadata Integration Service (OMIS) is running three integation connectors and they are all failing because the directories (folders) that they are supposed to be monitoring do not exist.  As the data onboarding pipeline is set up in this lab, the directories will get created and we will be able to restart the connectors to get them working.

In [ ]:

OakDeneLandingAreaConnectorName   = 'OakDeneLandingAreaFilesMonitor'
OldMarketLandingAreaConnectorName = 'OldMarketLandingAreaFilesMonitor'
DataLakeDirectoryConnectorName    = 'DropFootClinicalTrialResultsFolderMonitor'


Below are the names of the directories that the three integration connectors are monitoring.  The integration connector monitoring the files in the data lake directory is maintaining the last update time for the [DataFolder](https://egeria.odpi.org/open-metadata-publication/website/modelling-technology/) asset that represents the collection of measurements received from all hospitals for the clinical trial.

In [ ]:

landingAreaDirectory      = fileSystemRoot + '/landing-area/hospitals'
dataLakeDirectory         = fileSystemRoot + '/data-lake/research/clinical-trials/drop-foot/weekly-measurements'

OakDeneLandingDirectory   = landingAreaDirectory + '/oak-dene/clinical-trials/drop-foot'
OldMarketLandingDirectory = landingAreaDirectory + '/old-market/clinical-trials/drop-foot'


The files that will provisioned into these directories are located in the sample data that is in the Egeria distribution.

In [ ]:

OakDeneSourceFolder   = egeriaSampleDataRoot + '/sample-data/oak-dene-drop-foot-weekly-measurements'
OldMarketSourceFolder = egeriaSampleDataRoot + '/sample-data/old-market-drop-foot-weekly-measurements'



At this time there are no assets catalogued for either the landing area or the weekly measurements directory.

----

In [ ]:

assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*landing.*")
assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*drop-foot/weekly-measurements.*")


----
## Setting up the file transfer into the landing area

The directories (folders) that the integration connectors are configured to monitor could be created using a file system command.
In this lab, however, the creation of a landing area folder will occur when the first file is received from the corresponding hospital.
This is simply to avoid needing to provide this notebook with access to the file system.

We are going to use a
[provisioning governance action service](https://egeria.odpi.org/open-metadata-implementation/frameworks/governance-action-framework/docs/provisioning-governance-service.html) called
[Move/Copy File Governance Action Service](https://egeria.odpi.org/open-metadata-publication/website/connector-catalog/move-copy-file-provisioning-governance-action-service.html)
to simulate the file transfer from a hospital to its folder in the landing zone.
This service runs in a governance engine that is supported by the
[Governance Action Open Metadata Engine Service (OMES)](https://egeria.odpi.org/open-metadata-implementation/engine-services/governance-action/).

It is possible to validate and understand the function of the governance action service
through an API call to the Governance Action OMES running an engine host server such as `governDL01`.
This API call is designed to support tools and others services configuring governance action services.

Governance action services are implemented as a connector, which means they are initialized with a
[connection](https://egeria.odpi.org/open-metadata-implementation/frameworks/open-connector-framework/docs/concepts/connection.html) object.
This includes configuration properties that can be used to control its behavior.
`Move/Copy File` is highly configurable. 

----

In [ ]:

moveCopyFileProviderClassName = "org.odpi.openmetadata.adapters.connectors.governanceactions.provisioning.MoveCopyFileGovernanceActionProvider"

print("Move/Copy File Governance Action Service:")
validateGovernanceActionEngineConnector(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId, moveCopyFileProviderClassName)


----

To simulate the secure file transfer from the hospital to the landing area, we will configure
`Move/Copy File`
so it expects that the name of the source file to copy will be supplied when it is called (rather than using the catalog entry for the
source file) and it does not create lineage.

----

In [ ]:

ftpGovernanceServiceName = "ftp-governance-action-service"
ftpGovernanceServiceDisplayName = "FTP Governance Action Service"
ftpGovernanceServiceDescription = "Simulates FTP from an external party."
ftpGovernanceServiceProviderClassName = moveCopyFileProviderClassName
ftpGovernanceServiceConfigurationProperties = {
        "noLineage" : ""
    }
ftpGovernanceServiceRequestType = "copy-file"


----

This governance action service
runs in a governance engine supported by the Governance Action Open Metadata Engine Services (OMES).
Figure 5 shows the three governance engines configured in the `governDL01` engine host server during the
[Configuring Egeria Servers Lab](../egeria-server-config.ipynb).

![Figure 5](../images/engine-host.png)
> **Figure 5:** Governance Engines for governDL01

The command below queries the status of each governance engine running in `governDL01`.
The governance action services that will support the onboarding of files for clinical trials will run in the `AssetGovernance`
governance engine.  The other two governance engines are the subject of the [Improving Data Quality](../administration-labs/open-discovery-config.ipynb) lab.

----

In [ ]:

printGovernanceEngineStatuses(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId)


----
The status code `ASSIGNED` means that the governance engine was listed in Engine Host's configuration
document - ie the governance engine was assigned to this server - but Engine Host has not been
able to retrieve the configuration for the governance engine from the governance metadata server (`cocoMDS2`).

The next step in the lab is to add configuration for the governance engine to `cocoMDS2` until the
`AssetGovernance` governance engine is running.  The first step is to add a GovernanceEngine definition
to the metadata server for `AssetGovernance`.  

----

In [ ]:
assetGovernanceEngineName = "AssetGovernance"
assetGovernanceEngineDisplayName = "Asset Governance Action Engine"
assetGovernanceEngineDescription = "Monitors, validates and enriches metadata relating to assets."

assetGovernanceEngineGUID = createGovernanceEngine(cocoMDS2Name,
                                                   cocoMDS2PlatformName,
                                                   cocoMDS2PlatformURL,
                                                   erinsUserId,
                                                   "GovernanceActionEngine",
                                                   assetGovernanceEngineName,
                                                   assetGovernanceEngineDisplayName,
                                                   assetGovernanceEngineDescription)

if (assetGovernanceEngineGUID):
    print (" ")
    print ("The guid for the " + assetGovernanceEngineName + " governance engine is: " + assetGovernanceEngineGUID)
    print (" ")

print ("Done. ")    

----

Now the governance engine is defined, its definition is sent to `governDL01` in an event
and the engine's status moves to `CONFIGURING`.
Notice more descriptive information is returned with the status.

----

In [ ]:

waitForConfiguringGovernanceEngine(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId, assetGovernanceEngineName)

printGovernanceEngineStatuses(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId)


----

Now we add the description of `Move/Copy File` to `cocoMDS2`.  There are two parts to
registering a governance action service.  The first is to create a GovernanceService definition that identifies
the implementation of the governance action service and the second part registers this
GovernanceService definition with the GovernanceEngine definition.
This registration maps one or more of the governance engine's request types, along with the default request parameters to
the GovernanceService definition.  This mapping is used to translate a request to the governance engine into an
invocation of a governance action service.

Figure 6 shows the structure of the resulting definitions for a governance engine.  A governance action service may be
registered with multiple governance engines, using the same or different request types.

![Figure 6](../images/governance-action-request-type.png)
> **Figure 6:** Structure of the governance services within a governance engine

This next code issues the calls to create a definition of the GovernanceService and add it to the AssetGovernance
GovernanceEngine definition.

----

In [ ]:
governanceServiceGUID = createGovernanceService(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                ftpGovernanceServiceName,
                                                ftpGovernanceServiceDisplayName,
                                                ftpGovernanceServiceDescription,
                                                ftpGovernanceServiceProviderClassName,
                                                ftpGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + ftpGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(cocoMDS1Name,
                                        cocoMDS1PlatformName,
                                        cocoMDS1PlatformURL,
                                        petersUserId,
                                        assetGovernanceEngineGUID,
                                        governanceServiceGUID,
                                        ftpGovernanceServiceRequestType)
    print ("Service registered as: " + ftpGovernanceServiceRequestType)
    print (" ")
    

----

When at least one governance service is registered with the governance engine, the status of the governance engine in
`governDL01` moves to `RUNNING` and it is possible to see the list of supported request types for the governance engine. 

----

In [ ]:

waitForRunningGovernanceEngine(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId, assetGovernanceEngineName)

printGovernanceEngineStatuses(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId)


----

With the governance action service defined in a running governance engine, all that remains is to call it.

Any defined governance action service can be called directly through the Governance Engine OMAS,
which in this lab is running in the `cocoMDS2` metadata server.  This call results in an event being sent to all
engine hosts running the named governance engine.  The first one that claims it gets to run it.  In this lab
there is only one engine host and so the request will run on `governDL01`.

This governance action does not take long to run so you should soon see its status as completed.

----

In [ ]:

def addFileToLandingArea(sourceFolder, destinationFolder, weekNumber, qualifiedName):
    sourceFileName = sourceFolder + "/" + "week" + weekNumber + ".csv"
    requestParameters = {
        "sourceFile" : sourceFileName,
        "destinationFolder" : destinationFolder
    }
    governanceActionGUID = None
    governanceActionGUID = initiateGovernanceAction(cocoMDS2Name,
                                                    cocoMDS2PlatformName,
                                                    cocoMDS2PlatformURL,
                                                    erinsUserId,
                                                    assetGovernanceEngineName,
                                                    qualifiedName,
                                                    ftpGovernanceServiceRequestType,
                                                    requestParameters)
    waitForRunningGovernanceAction(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, governanceActionGUID)
    printGovernanceAction(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, governanceActionGUID)

addFileToLandingArea(OakDeneSourceFolder, OakDeneLandingDirectory, "1", "FTP Oak Dene Week 1")

----
We now have the ability to provision new files into either landing area as shown in Figure 7.

![Phase 1](../images/automated-curation-scenario-1.png)
> **Figure 7:** Phase 1: Arrival of new files

----
Restarting the oak dene integration connector in the `exchangeDL01` integration daemon will cause it to
test again to see if the directory it is monitoring exists.

----

In [ ]:

restartIntegrationConnector(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId, "files-integrator", OakDeneLandingAreaConnectorName)


----

If we check the status of the integration connectors again, you can see this connector is now running.

----

In [ ]:

getIntegrationDaemonStatus(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId)


----

The connector will begin cataloguing any files placed in the directory.  We should be able to see the `week1.csv` file in the catalog along with catalog entries for each of the nested directories it sits in. If no assets are shown wait a short while and run the cell again and they will appear.

----

In [ ]:

assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*landing.*")


----

The call below captures the full path name of the landing area directory by extracting it from the catalogued asset for the this directory.
This will be used when configuring a later part of the onboarding process.

----

In [ ]:

landingAreaDirectoryName = assetOwnerFindAssetQualifiedName(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*" + landingAreaDirectory)

print(landingAreaDirectoryName)

----
The action of the integration connector has been to catalog the new file and the folders above it.
The catalog definition is minimal, consisting of just what can be gleaned from the file system.

It is possible to provide the integration connector with a template metadata element to copy when it
is cataloguing files.   This template can include classifications such as zones, origin, confidentiality and attachments such as
connections, schemas and lineage mappings. (There is more information on templates in the
[Egeria website](https://egeria.odpi.org/open-metadata-publication/website/cataloging-assets/templated-cataloging.html)).

Templates are used to set up values for the assets created by the connector that are always the same.
In this example, all of the files that this connector encounters are from the Oak Dene Hospital so we can use a template to set up the
file's origin and the connection information needed to access the file.

Including this type of detail in the asset means that no-one has to remember that this landing area folder was used by
the Oak Dene Hospital and it simplifies the downstream cataloguing.
Other values that are useful to set up in a template are any licenses for the file, schema information, zones, known lineage to this
directory, plus other classifications.  Information about the types of information that can be attached to an
asset are available on the [Egeria website](https://egeria.odpi.org/open-metadata-publication/website/cataloging-assets/asset-catalog-contents.html).



## Working with templates

If you know the class name of an integration connector's provider, it is possible to check if the connector is of the right type for an integration service.  This function also returns full details of the connector type, which often includes descriptive information as well as the configuration properties that it supports.

The connectors configured in the Files Integrator OMIS are shown in figure 8:

![Figure 8](../images/integration-daemon.png)
> **Figure 8:** exchangeDL01 with its connectors

The class names of these integration connectors' providers can be seen in the connection object embedded in the error message displayed with the connectors' status.

The commands below request that the Files Integrator OMIS service validate and return the connector type for each of these connectors.

In [ ]:
dataFilesMonitorProviderClassName = "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFilesMonitorIntegrationProvider"
dataFolderMonitorProviderClassName = "org.odpi.openmetadata.adapters.connectors.integration.basicfiles.DataFolderMonitorIntegrationProvider"

print("Data Files Monitor Integration Connector Type:")
validateIntegrationConnector(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, "files-integrator", petersUserId, dataFilesMonitorProviderClassName)

print("")
print("Data Folder Monitor Integration Connector Type:")
validateIntegrationConnector(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, "files-integrator", petersUserId, dataFolderMonitorProviderClassName)

----
Both connectors supports the `templateQualifiedName` and the `allowCatalogDelete` configuration properties.  If you are curious
about their meaning, review the definitions in the connector catalog:

* [Data Files Monitor Integration Connector](https://egeria.odpi.org/open-metadata-publication/website/connector-catalog/data-files-monitor-integration-connector.html)
* [Data Folder Monitor Integration Connector](https://egeria.odpi.org/open-metadata-publication/website/connector-catalog/data-folder-monitor-integration-connector.html)

We are going to set the `templateQualifiedName` with the qualified name of an asset that has the origin set up for the each of the appropriate originating hospitals and membership of the default governance zone of "quarantine".
This zone means that the asset is still being set up and it is not visible to the data lake users.

----
The commands below create the template assets for each of the hospitals. In addition, we will create a third template for the asset created to
represent the file when it is stored in the data lake folder.  It will set up Tanya Tidie as the owner of the files and marks the transition
of ownership from the data lake oeprations team to the clinical trials team.  It also has the zone membership set to default
governance zone of "quarantine".

This third template will be incorporated into the flow when we configure the governance action service that moves the file from the landing area to the data lake.

In [ ]:
newAssetZone    = "quarantine"

t1AssetName     = "Oak Dene Template"
t1QualifiedName = "template:clinical-trials:drop-foot:weekly-measurements:oak-dene"
t1DisplayName   = "Drop Foot Clinical Trial Measurements Template Asset: Oak Dene"
t1Description   = "Template asset for Oak Dene Hospital."
t1Contact       = "Robbie Records"
t1Dept          = "Drop Foot Research Centre"
t1Org           = "Oak Dene Hospital"

t2AssetName     = "Old Market Template"
t2QualifiedName = "template:clinical-trials:drop-foot:weekly-measurements:old-market"
t2DisplayName   = "Drop Foot Clinical Trial Measurements Template Asset: Old Market"
t2Description   = "Template asset for Old Market Hospital."
t2Contact       = "TBD"
t2Dept          = "DFRG1F6"
t2Org           = "Old Market Hospital"


template1guids = assetOwnerCreateCSVAsset(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, t1DisplayName, t1Description, t1QualifiedName)
template1guid = getLastGUID(template1guids)
addOrigin(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, t1AssetName, template1guid, t1Contact, t1Dept, t1Org)
addZones(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, petersUserId, t1AssetName, template1guid, [newAssetZone])

template2guids = assetOwnerCreateCSVAsset(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, t2DisplayName, t2Description, t2QualifiedName)
template2guid = getLastGUID(template2guids)
addOrigin(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, t2AssetName, template2guid, t2Contact, t2Dept, t2Org)
addZones(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, petersUserId, t2AssetName, template2guid, [newAssetZone])

t3AssetName     = "Data Lake Measurements Template"
t3QualifiedName = "template:clinical-trials:drop-foot:weekly-measurements:data-lake"
t3DisplayName   = "Drop Foot Clinical Trial Measurements Template Asset: Data Lake"
t3Description   = "Template asset for Data Lake destination folder."
t3Owner         = "tanyatidie"
t3OwnerType     = "USER_ID"

template3guids = assetOwnerCreateCSVAsset(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, t3DisplayName, t3Description, t3QualifiedName)
template3guid = getLastGUID(template3guids)
addOwner(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, petersUserId, t3AssetName, template3guid, t3Owner, t3OwnerType)
addZones(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, petersUserId, t3AssetName, template3guid, [newAssetZone])

print ("\n\nNewTemplate Assets:")
assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*Template.*")

----

The next commands reconfigure the integration connectors that are monitoring for new files in the landing area to use the templates ...

---

In [ ]:

t1ConnectorName = "OakDeneLandingAreaFilesMonitor"
t1ConfigurationProperties = {
    "templateQualifiedName" : t1QualifiedName
}

t2ConnectorName = "OldMarketLandingAreaFilesMonitor"
t2ConfigurationProperties = {
    "templateQualifiedName" : t2QualifiedName
}

updateConnectorConfigurationProperties(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, "files-integrator", petersUserId, t1ConnectorName, t1ConfigurationProperties)
getIntegrationConnectorConfigProperties(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, "files-integrator", petersUserId, t1ConnectorName)

updateConnectorConfigurationProperties(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, "files-integrator", petersUserId, t2ConnectorName, t2ConfigurationProperties)
getIntegrationConnectorConfigProperties(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, "files-integrator", petersUserId, t2ConnectorName)


----

Now lets add the file for week 2 into the landing area.

----

In [ ]:

addFileToLandingArea(OakDeneSourceFolder, OakDeneLandingDirectory, "2", "FTP Oak Dene Week 2")
 

----

The newly catalogued file will show that the origin and zones are set up.

----

In [ ]:

assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*landing-area/hospitals/oak-dene/clinical-trials/drop-foot/week.*")


----
We are now cataloguing files arriving in the landing folders and their origin is set up - and so phase 2 shown in figure 9 is complete.


![Phase 2](../images/automated-curation-scenario-2.png)
> **Figure 9:** Phase 2: Creating assets for newly arrived files with a template


At this point, the integration daemon supports the onboarding of data from either hospital.
The next step is to detect the creation of each file's catalogued asset to drive the process that moves the file into the data lake directory,
and creates a catalog asset for its new location, along with lineage to show the onboarding path of the file.

## Listening for new assets

The watchdog governance action services are responsible for listening for events that indicate specific types of activity and then acting on it.
They can initiate
a [governance action](https://egeria.odpi.org/open-metadata-implementation/access-services/governance-engine/docs/concepts/governance-action.html),
a [governance action process](https://egeria.odpi.org/open-metadata-implementation/access-services/governance-engine/docs/concepts/governance-action-process.html) or
an [incident report](https://egeria.odpi.org/open-metadata-implementation/access-services/governance-engine/docs/concepts/incident-report.html).

For this onboarding process, we are going to set up a watchdog governance action service called `GenericFolderWatchdog` to
listen for assets that are catalogued and linked to the landing area directory.

Below is the definition of this governance action service.

----

In [ ]:

genericFolderWatchdogProviderClassName = "org.odpi.openmetadata.adapters.connectors.governanceactions.watchdog.GenericFolderWatchdogGovernanceActionProvider"

print("Generic Folder Watchdog Governance Action Service:")
validateGovernanceActionEngineConnector(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId, genericFolderWatchdogProviderClassName)


----

The calls below defines the governance action service and registers it with the `AssetGovernance` Governance Engine.

----

In [ ]:
watchdogGovernanceServiceName = "new-measurements-watchdog-governance-action-service"
watchdogGovernanceServiceDisplayName = "New Measurements Watchdog Governance Action Service"
watchdogGovernanceServiceDescription = "Initiates a governance action process when a new weekly measurements file arrives."
watchdogGovernanceServiceProviderClassName = genericFolderWatchdogProviderClassName
watchdogGovernanceServiceConfigurationProperties = {}
watchdogGovernanceServiceRequestType = "nested-in-folder"
watchdogGovernanceServiceProcessToCall = "governance-action-process:clinical-trials:drop-foot:weekly-measurements:onboarding"

governanceServiceGUID = createGovernanceService(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                watchdogGovernanceServiceName,
                                                watchdogGovernanceServiceDisplayName,
                                                watchdogGovernanceServiceDescription,
                                                watchdogGovernanceServiceProviderClassName,
                                                watchdogGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + watchdogGovernanceServiceDisplayName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(cocoMDS1Name,
                                        cocoMDS1PlatformName,
                                        cocoMDS1PlatformURL,
                                        petersUserId,
                                        assetGovernanceEngineGUID,
                                        governanceServiceGUID,
                                        watchdogGovernanceServiceRequestType)
    print ("Service registered as: " + watchdogGovernanceServiceRequestType)
    print (" ")
    


----

Now lets start it running in the Engine Host server `governDL01` by initiating a governance action.

----

In [ ]:

requestParameters = {
        "interestingTypeName" : "DataFile",
        "folderName" : landingAreaDirectoryName,
        "actionTargetName" : "sourceFile",
        "newElementProcessName" : watchdogGovernanceServiceProcessToCall
    }
qualifiedName = "Listen for newly catalogued CSV Files in directory " + landingAreaDirectory

governanceActionGUID = None
governanceActionGUID = initiateGovernanceAction(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                assetGovernanceEngineName,
                                                qualifiedName,
                                                watchdogGovernanceServiceRequestType,
                                                requestParameters)


----

With the watchdog in place, any new files added to the metadata repository will trigger a governance action process called `governance-action-process:clinical-trials:drop-foot:weekly-measurements:onboarding`.

![Phase 3](../images/automated-curation-scenario-3.png)
> **Figure 10:** Phase 3: Triggering provisioning of the file in to the data lake

## Creating a governance action process for new assets

A governance action process defines a flow of calls to governance action services
that are linked together by the guards that they produce when they run.
The guards produced by one governance action service invocation determine which governance action service runs next.

A governance action process is represented by a metadata element of type [GovernanceActionProcess](https://egeria.odpi.org/open-metadata-publication/website/open-metadata-types/0462-Governance-Action-Types.html).
This element gives the process a unique name and the anchor point to connect it into the governance action definitions.

----

In [ ]:

qualifiedName = watchdogGovernanceServiceProcessToCall
displayName   = "Drop Foot Onboard Weekly Measurement Files"
description   = "Ensures that new weekly drop foot measurement files from the hospitals are correctly catalogued in the data lake."
technicalName = "DFOBWKLY01"
technicalDescription = """ This process performs the follow function:
 1) The physical file is moved to the data lake and renamed 
 2) A new asset is created for the new file 
 3) Lineage is created between the orginal file asset and the new file asset
 4) The owner and origin are assigned
 5) The governance zones are assigned to make the new asset visible to the research team."""

governanceActionProcessGUID = createGovernanceActionProcess(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, qualifiedName, displayName, description, technicalName, technicalDescription)


----

Added to the governance action process are the definitions of which type of governance actions should run when the process is initiated.  These definitions are represented by
[GovernanceActionType](https://egeria.odpi.org/open-metadata-publication/website/open-metadata-types/0462-Governance-Action-Types.html) metadata elements.  They are linked together to define which governance action is started depending on the
guards produced by the previous governance action.

Each GovernanceActionType links to a GovernanceActionService configured in the GovernanceEngine.

The first step in the process is to move the file from the landing area to the data lake folder.

![Phase 4](../images/automated-curation-scenario-4.png)
> **Figure 11:** Phase 4: Move the file into the data lake

For this we use a second instance of the `Move/Copy File` Governance Action Service.
This instance will be set up to move the file, produce lineage and change the resulting filename so that the files are sequenced according to their arrival.  For example:
 * DropFoot_000001.csv
 * DropFoot_000002.csv

This aids the time-based loading of the files into a database by ensuring any corrections to the readings are applied in the
correct order if a hospital sends a correction file at a later date.

----

In [ ]:

prGovernanceServiceName = "provision-weekly-measurements-governance-action-service"
prGovernanceServiceDisplayName = "File Provisioning Governance Action Service"
prGovernanceServiceDescription = "Provisions weekly measurement files from the landing area to the data lake."
prGovernanceServiceProviderClassName = moveCopyFileProviderClassName
prGovernanceServiceConfigurationProperties = {
        "targetFileNamePattern" : "DropFoot_{1, number,000000}.csv",
        "destinationFolder" : dataLakeDirectory
    }
prGovernanceServiceRequestType = "move-file"

governanceServiceGUID = createGovernanceService(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                prGovernanceServiceName,
                                                prGovernanceServiceDisplayName,
                                                prGovernanceServiceDescription,
                                                prGovernanceServiceProviderClassName,
                                                prGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + prGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(cocoMDS1Name,
                                        cocoMDS1PlatformName,
                                        cocoMDS1PlatformURL,
                                        petersUserId,
                                        assetGovernanceEngineGUID,
                                        governanceServiceGUID,
                                        prGovernanceServiceRequestType)
    print ("Service registered as: " + prGovernanceServiceRequestType)
    print (" ")



----

This governance action service is linked into the process via the following governance action type.
Notice that the template created for cataloguing the files as they move into the data lake directory is set up in the `destinationFileTemplateQualifiedName` requestParameter as required for phase 5.

![Phase 5](../images/automated-curation-scenario-5.png)
> **Figure 12:** Phase 5: Set up the template for newly arrived files in the data lake

----

In [ ]:

prGovernanceServiceType = "provision-weekly-measurements-governance-action-type"
prSupportedGuards = ["provisioning-complete", "provisioning-failed"]
prGovernanceServiceRequestParameters = {
        "destinationFileTemplateQualifiedName" : t3QualifiedName,
    }

prActionTypeGUID = None
prActionTypeGUID = createGovernanceActionType(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, assetGovernanceEngineGUID, prGovernanceServiceType, prSupportedGuards, prGovernanceServiceRequestType, prGovernanceServiceRequestParameters) 

if prActionTypeGUID:
    setupFirstActionType(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, governanceActionProcessGUID, prActionTypeGUID, False)


----

Now that the files are in the correct folder with the appropiate file name and a new catalog entry has been created, it is time to
add the last touches to the catalog entry before it is published to the data lake users.
This is phase 6 of the onboarding process.

![Phase 6](../images/automated-curation-scenario-6.png)
> **Figure 12:** Phase 6: Extend the governance action process to enrich the asset

The template used to create the catalog was able to set up the owner of the file because it is the same for each file that is moved into this directory.
However, the origin is variable, depending on which hospital send the original file.
To set the origin, we are going to use the
[Origin Seeker Remediation Governance Action Service](https://egeria.odpi.org/open-metadata-publication/website/connector-catalog/origin-seeker-remediation-governance-action-service.html)
that is able to navigate back through the lineage relationship created by the `Move/Copy File` governance action service to pick up the origin
from the landing area asset.

Below is the definition of `OriginSeeker`.

----

In [ ]:

originSeekerProviderClassName = "org.odpi.openmetadata.adapters.connectors.governanceactions.remediation.OriginSeekerGovernanceActionProvider"

print("Origin Seeker Remediation Governance Action Service:")
validateGovernanceActionEngineConnector(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId, originSeekerProviderClassName)


----

The calls below add `OriginSeeker` to the `AssetGovernance` governance engine, creates a governance action type and links it to the
governance action type for `Move/Copy File`.

----

In [ ]:
osGovernanceServiceName = "origin-seeker-governance-action-service"
osGovernanceServiceDisplayName = "Locate and Set Origin Governance Action Service"
osGovernanceServiceDescription = "Navigates back through the lineage relationships to locate the origin classification(s) from the source(s) and sets it on the requested asset if the origin is unique."
osGovernanceServiceProviderClassName = originSeekerProviderClassName
osGovernanceServiceConfigurationProperties = {}
osGovernanceServiceRequestType = "seek-origin"

governanceServiceGUID = createGovernanceService(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                osGovernanceServiceName,
                                                osGovernanceServiceDisplayName,
                                                osGovernanceServiceDescription,
                                                osGovernanceServiceProviderClassName,
                                                osGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + osGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(cocoMDS1Name,
                                        cocoMDS1PlatformName,
                                        cocoMDS1PlatformURL,
                                        petersUserId,
                                        assetGovernanceEngineGUID,
                                        governanceServiceGUID,
                                        osGovernanceServiceRequestType)
    print ("Service registered as: " + osGovernanceServiceRequestType)
    print (" ")

osPreviousGuard = "provisioning-complete"
osGovernanceServiceType = "origin-seeker-measurements-governance-action-type"
osSupportedGuards = [ "origin-assigned", "origin-already-assigned", "multiple-origins-detected", "no-origins-detected", "no-targets-detected", "multiple-targets-detected", "origin-seeking-failed"]
osGovernanceServiceRequestParameters = {
        "destinationFileTemplateQualifiedName" : t3QualifiedName
    }

osActionTypeGUID = createGovernanceActionType(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, assetGovernanceEngineGUID, osGovernanceServiceType, osSupportedGuards, osGovernanceServiceRequestType, osGovernanceServiceRequestParameters) 

setupNextActionType(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, prActionTypeGUID, osActionTypeGUID, osPreviousGuard, True, True)

----

Once the origin is set, the asset is completely defined and it is ready to be consumed by the data lake users.
The last governance action service called `ZonePublisher` sets the zone membership for the asset so that it becomes visible to the
data lake users.

Below is the definition of `ZonePublisher`.

----

In [ ]:

zonePublisherProviderClassName = "org.odpi.openmetadata.adapters.connectors.governanceactions.remediation.ZonePublisherGovernanceActionProvider"

print("Zone Publisher Remediation Governance Action Service:")
validateGovernanceActionEngineConnector(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId, zonePublisherProviderClassName)


----

The calls below add `ZonePublisher` to the `AssetGovernance` governance engine, creates a governance action type and links it to the
governance action type for `OriginSeeker`.

----

In [ ]:

zpGovernanceServiceName = "zone-publisher-governance-action-service"
zpGovernanceServiceDisplayName = "Update Asset's Zone Membership Governance Action Service"
zpGovernanceServiceDescription = "Set up the zone membership for one or more assets supplied as action targets."
zpGovernanceServiceProviderClassName = zonePublisherProviderClassName
zpGovernanceServiceConfigurationProperties = {
    "publishZones" : "data-lake,clinical-trials"
}
zpGovernanceServiceRequestType = "set-zone-membership"

governanceServiceGUID = createGovernanceService(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                zpGovernanceServiceName,
                                                zpGovernanceServiceDisplayName,
                                                zpGovernanceServiceDescription,
                                                zpGovernanceServiceProviderClassName,
                                                zpGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + zpGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(cocoMDS1Name,
                                        cocoMDS1PlatformName,
                                        cocoMDS1PlatformURL,
                                        petersUserId,
                                        assetGovernanceEngineGUID,
                                        governanceServiceGUID,
                                        zpGovernanceServiceRequestType)
    print ("Service registered as: " + zpGovernanceServiceRequestType)
    print (" ")

zpPreviousGuard = "origin-assigned"
zpGovernanceServiceType = "zone-publisher-measurements-governance-action-type"
zpSupportedGuards = [ "zone-assigned","no-zones-detected","no-targets-detected", "zone-publishing-failed"]
zpGovernanceServiceRequestParameters = {
        "destinationFileTemplateQualifiedName" : t3QualifiedName
    }

zpActionTypeGUID = createGovernanceActionType(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, assetGovernanceEngineGUID, zpGovernanceServiceType, zpSupportedGuards, zpGovernanceServiceRequestType, zpGovernanceServiceRequestParameters) 

setupNextActionType(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, osActionTypeGUID, zpActionTypeGUID, zpPreviousGuard, True, True)


----

We can check that these services are now registered with the `AssetGovernance` governance engine.

----

In [ ]:

printGovernanceEngineStatuses(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId)


----

Lets provision week 3 to see if the process is triggered and the file is moved to the data lake folder.

----

In [ ]:

addFileToLandingArea(OakDeneSourceFolder, OakDeneLandingDirectory, "3", "FTP Oak Dene Week 3")


----

The file is created in the data lake folder with all of the correct attributes.

----

In [ ]:

assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*clinical-trials/drop-foot/weekly-measurements.*")


----

The file in the landing area is gone.

----

In [ ]:

assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*landing-area/hospitals/oak-dene/clinical-trials/drop-foot/.*")


----
## Running the complete onboarding pipeline

The finale of this lab is to show the complete pipeline running.  To do this we will use the files from both Oak Dene Hospital and the Old Market Hospital. First it is necessary to delete the Oak Dene files used in the experiments when setting up the pipeline.  This can be done with a `Move/Copy File` governance action service configured to delete named files.

In [ ]:
dlGovernanceServiceName = "delete-files-governance-action-service"
dlGovernanceServiceDisplayName = "File Delete Governance Action Service"
dlGovernanceServiceDescription = "Removes weekly measurement files from the landing area."
dlGovernanceServiceProviderClassName = moveCopyFileProviderClassName
dlGovernanceServiceConfigurationProperties = {
        "noLineage" : ""
    }
dlGovernanceServiceRequestType = "delete-file"

governanceServiceGUID = createGovernanceService(cocoMDS2Name,
                                                cocoMDS2PlatformName,
                                                cocoMDS2PlatformURL,
                                                erinsUserId,
                                                "GovernanceActionService",
                                                dlGovernanceServiceName,
                                                dlGovernanceServiceDisplayName,
                                                dlGovernanceServiceDescription,
                                                dlGovernanceServiceProviderClassName,
                                                dlGovernanceServiceConfigurationProperties)

if (governanceServiceGUID):
    print (" ")
    print ("The guid for the " + dlGovernanceServiceName + " governance service is: " + governanceServiceGUID)
    registerGovernanceServiceWithEngine(cocoMDS1Name,
                                        cocoMDS1PlatformName,
                                        cocoMDS1PlatformURL,
                                        petersUserId,
                                        assetGovernanceEngineGUID,
                                        governanceServiceGUID,
                                        dlGovernanceServiceRequestType)
    print ("Service registered as: " + dlGovernanceServiceRequestType)
    print (" ")


def removeFileFromLandingArea(sourceFolder, weekNumber, qualifiedName):
    sourceFileName = sourceFolder + "/" + "week" + weekNumber + ".csv"
    requestParameters = {
        "sourceFile" : sourceFileName
    }
    governanceActionGUID = None
    governanceActionGUID = initiateGovernanceAction(cocoMDS2Name,
                                                    cocoMDS2PlatformName,
                                                    cocoMDS2PlatformURL,
                                                    erinsUserId,
                                                    assetGovernanceEngineName,
                                                    qualifiedName,
                                                    dlGovernanceServiceRequestType,
                                                    requestParameters)
    waitForRunningGovernanceAction(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, governanceActionGUID)
    printGovernanceAction(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, erinsUserId, governanceActionGUID)

removeFileFromLandingArea(OakDeneLandingDirectory, "1", "Remove Oak Dene Week 1")
removeFileFromLandingArea(OakDeneLandingDirectory, "2", "Remove Oak Dene Week 2")


----

The code below uses the `Move/Copy File` governance action service to add the first five weeks files from each hospital into their landing area.

----

In [ ]:

addFileToLandingArea(OldMarketSourceFolder, OldMarketLandingDirectory, "1", "FTP Old Market Hospital Week 1")
addFileToLandingArea(OakDeneSourceFolder,   OakDeneLandingDirectory,   "1", "FTP Oak Dene Hospital Week 1")
addFileToLandingArea(OldMarketSourceFolder, OldMarketLandingDirectory, "2", "FTP Old Market Hospital Week 2")
addFileToLandingArea(OakDeneSourceFolder,   OakDeneLandingDirectory,   "2", "FTP Oak Dene Hospital Week 2")
addFileToLandingArea(OldMarketSourceFolder, OldMarketLandingDirectory, "3", "FTP Old Market Hospital Week 3")
addFileToLandingArea(OakDeneSourceFolder,   OakDeneLandingDirectory,   "3", "FTP Oak Dene Hospital Week 3")
addFileToLandingArea(OldMarketSourceFolder, OldMarketLandingDirectory, "4", "FTP Old Market Hospital Week 4")
addFileToLandingArea(OakDeneSourceFolder,   OakDeneLandingDirectory,   "4", "FTP Oak Dene Hospital Week 4")
addFileToLandingArea(OldMarketSourceFolder, OldMarketLandingDirectory, "5", "FTP Old Market Hospital Week 5")
addFileToLandingArea(OakDeneSourceFolder,   OakDeneLandingDirectory,   "5", "FTP Oak Dene Hospital Week 5")


----

At this point, nothing has been received from Old Market hospital, so its landing folder is not defined.
This means its integration connector is in FAILED status.  The code below restarts all of the integration connnectors in `exchangeDL01`.

----

In [ ]:

restartIntegrationConnector(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId, "files-integrator", OldMarketLandingAreaConnectorName)
restartIntegrationConnector(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId, "files-integrator", OakDeneLandingAreaConnectorName)
restartIntegrationConnector(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId, "files-integrator", DataLakeDirectoryConnectorName)


----

When we check the status of the integration daemon, all connectors are running.

----

In [ ]:

getIntegrationDaemonStatus(exchangeDL01Name, exchangeDL01PlatformName, exchangeDL01PlatformURL, petersUserId)


----

This is the final status of the `AssetGovernance` governance engine.

----

In [ ]:

printGovernanceEngineStatuses(governDL01Name, governDL01PlatformName, governDL01PlatformURL, petersUserId)


----

From Peter Profile's perspective, he can see that all of the files have been created in the data lake folder and the landing area is empty ...

----

In [ ]:
assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*clinical-trials/drop-foot/weekly-measurements.*")

In [ ]:
oakDeneLandingAreaFullPath = "/" + landingAreaDirectoryName + "/oak-dene/clinical-trials/drop-foot"

os.listdir (oakDeneLandingAreaFullPath)

----

In [ ]:
oldMarketLandingAreaFullPath = "/" + landingAreaDirectoryName + "/old-market/clinical-trials/drop-foot"

os.listdir (oldMarketLandingAreaFullPath)

In [ ]:
monitorGovernanceActions(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL, petersUserId)

----

In [ ]:
assetOwnerPrintAssets(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL, petersUserId, ".*landing-.*")

----

<img src="https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/callie-quartile.png" style="float:right">

Finally we check that Callie Quartile, a data scientist working in the research team, is able to see the files from `cocoMDS3`:



In [ ]:

assetConsumerPrintAssets(cocoMDS3Name, cocoMDS3PlatformName, cocoMDS3PlatformURL, calliesUserId, ".*clinical-trials/drop-foot/weekly-measurements.*")


----
## Where to next

* [Improving Data Quality Lab](improving-data-quality-lab.ipynb) - follow Peter as he makes use of automated metadata discovery to detect quality errors in the
  measurement files from the hospitals.
* [Understanding an Asset](understanding-an-asset.ipynb) - work with 

----